<a href="https://colab.research.google.com/github/ridabayi/AutoMl/blob/main/AutoMl_LazyPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lazypredict
!pip install scikit-learn==0.23.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import lazypredict
from lazypredict.Supervised import LazyClassifier
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

sns.set_context('notebook')
sns.set_style('white')

In [ ]:
flight_delay = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0JD1EN/Airlines.csv")

In [ ]:
flight_delay.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [ ]:
flight_delay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           539383 non-null  int64 
 1   Airline      539383 non-null  object
 2   Flight       539383 non-null  int64 
 3   AirportFrom  539383 non-null  object
 4   AirportTo    539383 non-null  object
 5   DayOfWeek    539383 non-null  int64 
 6   Time         539383 non-null  int64 
 7   Length       539383 non-null  int64 
 8   Delay        539383 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 37.0+ MB


In [ ]:
for col in ['Airline', 'AirportFrom', 'AirportTo']:
    flight_delay[col] = flight_delay.loc[:, col].astype('category')

In [ ]:
flight_delay.isna().sum()

id             0
Airline        0
Flight         0
AirportFrom    0
AirportTo      0
DayOfWeek      0
Time           0
Length         0
Delay          0
dtype: int64

##Spliting Data

In [ ]:
print("Number of observations: ", flight_delay.shape[0])
print("Number of features: ", flight_delay.shape[1])

Number of observations:  539383
Number of features:  9


In [ ]:
X = flight_delay[:8000].drop(['id', 'Delay'], axis=1)
y = np.array(flight_delay['Delay'][:8000])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=0)

In [ ]:
print("Train set size: ", X_train.shape)
print("Test set size: ", X_test.shape)

Train set size:  (6000, 7)
Test set size:  (2000, 7)


In [ ]:
# Specify predictions=True returns test set predictions as a dataframe
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None, predictions=True)
models, preds = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:19<00:00,  1.47it/s]

[LightGBM] [Info] Number of positive: 2348, number of negative: 3652
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000326 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 763
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391333 -> initscore=-0.441711
[LightGBM] [Info] Start training from score -0.441711


## Evaluate performance

In [ ]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
NearestCentroid                    0.61               0.61     0.61      0.62   
RandomForestClassifier             0.63               0.59     0.59      0.62   
LGBMClassifier                     0.63               0.59     0.59      0.62   
AdaBoostClassifier                 0.64               0.58     0.58      0.61   
XGBClassifier                      0.62               0.58     0.58      0.61   
LabelSpreading                     0.64               0.58     0.58      0.61   
ExtraTreesClassifier               0.62               0.58     0.58      0.61   
LabelPropagation                   0.63               0.58     0.58      0.61   
KNeighborsClassifier               0.61               0.57     0.57      0.60   
BernoulliNB                        0.63               0.57     0.57      0.59   
LinearDiscriminantAnalysis  

In [ ]:
adm = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0JD1EN/adm_data.csv")

In [ ]:
# Check sample values
adm.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.50,4.50,9.65,1,0.92
1,2,324,107,4,4.00,4.50,8.87,1,0.76
2,3,316,104,3,3.00,3.50,8.00,1,0.72
3,4,322,110,3,3.50,2.50,8.67,1,0.80
4,5,314,103,2,2.00,3.00,8.21,0,0.65


In [ ]:
# Check column names
list(adm.columns)

['Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [ ]:
# Rename columns that have extra trailing whitespace
adm.rename(columns={"Chance of Admit ": "chance-of-adm", "LOR ": "LOR"}, errors="raise", inplace=True)

In [ ]:
# Check column type
adm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   chance-of-adm      400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


In [ ]:
# Convert variable 'Research' to take type 'category' because it's a binary variable
adm['Research'] = adm.loc[:, 'Research'].astype('category')

In [ ]:
# Check for NAs
adm.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
chance-of-adm        0
dtype: int64

In [ ]:
X = adm.drop(['Serial No.', 'chance-of-adm'], axis=1)
y = np.array(adm['chance-of-adm'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models, preds = reg.fit(X_train, X_test, y_train, y_test)

100%|██████████| 42/42 [00:02<00:00, 16.09it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000043 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 178
[LightGBM] [Info] Number of data points in the train set: 300, number of used features: 6
[LightGBM] [Info] Start training from score 0.726233
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [ ]:
print(models)

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
SGDRegressor                                 0.70       0.72  0.07        0.01
HuberRegressor                               0.70       0.72  0.07        0.02
LinearSVR                                    0.70       0.72  0.07        0.02
ElasticNetCV                                 0.70       0.72  0.07        0.13
LassoLarsIC                                  0.70       0.72  0.07        0.03
LassoLarsCV                                  0.70       0.72  0.07        0.03
LarsCV                                       0.70       0.72  0.07        0.06
LassoCV                                      0.70       0.72  0.07        0.06
BayesianRidge                                0.70       0.72  0.07        0.02
Ridge                                        0.70       0.72  0.07        0.02
RidgeCV                                      0.70   